In [3]:
!ls

'3D structures.sdf'   convertFASTAtoSDF.py
 ConvertSDF.ipynb    'drug sequences.fasta'
 README.txt	      extract.py
 __pycache__	      pharmacologically_active_extracted.csv
 all_extracted.csv    protein.fasta


In [4]:
import sys
import os
import pandas as pd
import shutil

In [37]:
def getAllSDFname(fileTXT):
    allName = []
    with open(fileTXT, 'r+', encoding="utf-8") as f:
        lines = f.readlines()
        for i in range(0, len(lines)):
            line = lines[i]
            # print (line)
            if line.startswith("> <DATABASE_ID"):
                ne = lines[i + 1][:-1]
                allName.append(str(ne))

    ret_df = pd.DataFrame({"All": allName})
    ret_file_path = fileTXT + ""
    ret_file_path =  "extract_" + ret_file_path.replace("-copy.txt", ".csv")
    ret_df.to_csv(ret_file_path, index=False)
    print(len(allName))
    return allName


def split_3D(allName, location_path, fileTXT):
    indexFile = 0
    close = 1

    file1 = open(fileTXT, 'r', encoding="utf-8")
    try:
        while indexFile < len(allName) + 1:
            line = file1.readline()
            if close == 1:
                if indexFile >= len(allName):
                    break
                # Write file .txt
                file2 = open(
                    location_path + allName[indexFile] + ".txt", "w", encoding="utf-8")

                indexFile = indexFile + 1
                close = 0
            # if line.startswith("DB"): print(line)
            if not line.startswith("$$$$"):
                file2.write(line)
            else:
                file2.write("$$$$")
                file2.close()
                close = 1

                # Convert to sdf file
                thisFile = location_path + allName[indexFile - 1] + ".txt"
                base = os.path.splitext(thisFile)[0]
                os.rename(thisFile, base + ".sdf")

            # if line is empty
            # end of file is reached
            if not line:
                break
    finally:
        file1.close()

In [39]:
def getAllFASTAname(fileTXT):
    allName = []
    with open(fileTXT, 'r+', encoding="utf-8") as f:
        lines = f.readlines()
        for i in range(0, len(lines)):
            line = lines[i]
            if line.startswith(">drugbank"):
                # if '|' in lines[i][15:-1]: print(lines[i][15:22] + '*')
                name = lines[i].split()
                name = name[0].split("|")
                name = name[1]
                allName.append(str(name))

    ret_df = pd.DataFrame({"All": allName})
    ret_file_path = fileTXT + ""
    ret_file_path = "extract_" + ret_file_path.replace("-copy.txt", ".csv")
    ret_df.to_csv(ret_file_path, index=False)
    print(len(allName))
    return allName


def split_FASTA(allName, location_path, fileTXT):
    indexFile = -1
    resultDict = {}
    buff = ""
    file1 = open(fileTXT, 'r', encoding="utf-8")
    try:
        while 1:
            line = file1.readline()
            if line.startswith(">drugbank"):
                if(indexFile != -1):
                    if allName[indexFile] not in resultDict:
                        resultDict[allName[indexFile]] = buff
                    else:
                        if len(buff) > len(resultDict[allName[indexFile]]):
                            resultDict[allName[indexFile]] = buff
                    
                buff = line
                indexFile = indexFile + 1
            else:
                buff = buff + line
            # if line is empty
            # end of file is reached
            if not line:
                buff = buff + '\n' # for final file 
                if allName[indexFile] not in resultDict:
                    resultDict[allName[indexFile]] = buff
                else:
                    if len(buff) > len(resultDict[allName[indexFile]]):
                        resultDict[allName[indexFile]] = buff
                break
    finally:
        file1.close()
    print("Not duplicate: ", len(resultDict))
    for key in resultDict:
        file2 = open(location_path + key + ".txt", "w", encoding="utf-8")
        file2.write(resultDict[key][:-1])
        file2.close()
        
        # Convert to sdf file
        thisFile = location_path + key + ".txt"
        base = os.path.splitext(thisFile)[0]
        os.rename(thisFile, base + ".fasta")


def removeAllInFolder(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [35]:
def extract(folder_name, file_name):
    # Convert the copy file_name from .... to .txt
    base, ext = os.path.splitext(file_name)
    thisFile = base + "-copy" + ext
    shutil.copy2(file_name, thisFile)
    fileTXT = base + "-copy" + ".txt"
    os.rename(thisFile, fileTXT)
    
    # Create new folder to save result
    newpath = folder_name 
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    
    location_path = './'+ folder_name +'/'
    try:
        if ext == '.sdf':
            allName = getAllSDFname(fileTXT)
            # Read big file and split to small file and put in folder  split3D
            
            removeAllInFolder(location_path)
            split_3D(allName, location_path, fileTXT)
        elif ext == '.fasta':
            allName = getAllFASTAname(fileTXT)
            # Read big file and split to small file and put in folder  splitFASTA
            removeAllInFolder(location_path)
            split_FASTA(allName, location_path, fileTXT)

    finally:
        os.remove(fileTXT)


# python extract.py "3D structures_source.sdf"
# python extract.py "drug sequences.fasta"


In [40]:
# extract("3D structures_sdf", "3D structures.sdf")
extract("drug sequences_fasta", "drug sequences.fasta")
extract("protein_fasta", "protein.fasta")

360
Not duplicate:  233
5260
Not duplicate:  4867


In [ ]:
### Convert Fasta to SDF

In [43]:
# import openbabel
from openbabel import openbabel
import pandas as pd
from extract import removeAllInFolder


def getColumnData(file_name, column_name):
    fields = [column_name]
    df = pd.read_csv(file_name, skipinitialspace=True, usecols=fields)
    data = (list(df[column_name]))
    data = list(dict.fromkeys(data))  # remove duplicate
    return data


def convertFASTAtoSDF(list_file, folder_name):
    fromFolder = './'+folder_name+'/'
    resultFolder = './Convert_'+folder_name+'/'
    
    # Create new folder to save result
    newpath = resultFolder 
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    
    removeAllInFolder(resultFolder)
    for x in list_file:
        obConversion = openbabel.OBConversion()
        obConversion.SetInAndOutFormats("fasta", "sdf")

        obConversion.OpenInAndOutFiles(
            fromFolder + x + '.fasta', resultFolder + x + '.sdf')
        obConversion.Convert()
        obConversion.CloseOutFile()

In [44]:
allDrugs = getColumnData('all_extracted.csv', 'Drug IDs')
allSDF = getColumnData('extract_3D structures.csv', 'All')
allFASTA = getColumnData('extract_drug sequences.csv', 'All')

drugNotInSDF = []
for drug in allDrugs:
    if drug not in allSDF:
        drugNotInSDF = drugNotInSDF + [drug]

print("Drug not in SDF: ", len(drugNotInSDF))

drugInFasta = []
for drug in drugNotInSDF:
    if drug in allFASTA:
        drugInFasta = drugInFasta + [drug]

print("Drug not in SDF but In Fasta: ", len(drugInFasta))

print("Drug not in SDF and not in FASTA: ",
      len(drugNotInSDF) - len(drugInFasta))

convertFASTAtoSDF(drugInFasta, "drug sequences_fasta")


# conda install -c conda-forge openbabel


Drug not in SDF:  1644
Drug not in SDF but In Fasta:  187
Drug not in SDF and not in FASTA:  1457


In [45]:
allProt = getColumnData('all_extracted.csv', 'UniProt ID')
allProtFasta = getColumnData('extract_protein.csv', 'All')

ProtInFasta = []
for prot in allProt:
    if prot in allProtFasta:
        ProtInFasta = ProtInFasta + [prot]

print("Prot have in protein_fasta: ", len(ProtInFasta))
print("Prot not have in protein_fasta: ", len(allProt)-len(ProtInFasta))


convertFASTAtoSDF(ProtInFasta, "protein_fasta")

Prot have in protein_fasta:  4867
Prot not have in protein_fasta:  0


In [70]:
# Protein is in Convert_protein_fasta
# Ligand is in 3D structures_sdf + Convert_drug sequences_fasta

# all_active_pair                 in pharmacologically_active_extracted
# all active and in active = in all_extracted.csv


from shutil import copyfile

def load_uniprotid_drugid(file_path, prot_col, drug_col):
    df = pd.read_csv(file_path)
    protids = df[prot_col].values
    drugids = df[drug_col].values
    ret = [(protids[i],drugids[i]) for i in range(len(protids))]
#     print(ret)
    return ret


all_active_pair = load_uniprotid_drugid("pharmacologically_active_extracted.csv", "UniProt ID", "Drug IDs")
all_pair = load_uniprotid_drugid("all_extracted.csv", "UniProt ID", "Drug IDs")
all_inactive_pair = []

all_active_pair = list(dict.fromkeys(all_active_pair))  # remove duplicate
all_pair = list(dict.fromkeys(all_pair))  # remove duplicate

for pair in all_pair:
    if pair not in all_active_pair:
        all_inactive_pair = all_inactive_pair + [pair]
print(len(all_pair))
print(len(all_active_pair))
print(len(all_inactive_pair))
        
allSDF = getColumnData('extract_3D structures.csv', 'All')
allFASTA = getColumnData('extract_drug sequences.csv', 'All')

20992
5530
15462


In [71]:
for x in all_active_pair:
    print(x)
    prot = x[0]
    drug = x[1]
    if drug not in allSDF and drug not in allFASTA:
        print(drug)
        continue

    nameFolder = prot + "_CHEMBL" + drug
    
    # Create new folder to save result
    newpath = "dude1/" + nameFolder 
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    removeAllInFolder(newpath)
    
#     try:
    src = './Convert_protein_fasta/' + prot + ".sdf"
    dst = newpath + "/" + prot + ".sdf"
    copyfile(src, dst)

    src = None
    if drug in allSDF:
        src = './3D structures_sdf/' + drug + ".sdf"
    else:
        src = './Convert_drug sequences_fasta/' + drug + ".sdf"
    dst = dst = newpath + "/" + drug + ".sdf"
    copyfile(src, dst)

('P45059', 'DB00303')
('P00488', 'DB11300')
('P00488', 'DB11311')
('P00488', 'DB11571')
('P00488', 'DB13151')
DB13151
('P35228', 'DB08814')
('Q13423', 'DB09092')
('P50213', 'DB09092')
('Q9P0X4', 'DB00381')
('Q9P0X4', 'DB00568')
('Q9P0X4', 'DB00617')
('Q9P0X4', 'DB00909')
('Q9P0X4', 'DB01388')
('Q9P0X4', 'DB04841')
('P30542', 'DB00193')
('P30542', 'DB00201')
('P30542', 'DB00277')
('P30542', 'DB00640')
('P30542', 'DB00651')
('P30542', 'DB00806')
('P30542', 'DB01223')
DB01223
('P30542', 'DB01303')
DB01303
('P30542', 'DB01412')
('P00519', 'DB01254')
('P00519', 'DB04868')
('P00519', 'DB06616')
('P00519', 'DB08896')
('P00519', 'DB08901')
('P00519', 'DB12323')
('P12319', 'DB00043')
('P12319', 'DB00895')
DB00895
('P00451', 'DB00055')
('P00451', 'DB00100')
('P00451', 'DB11300')
('P00451', 'DB11312')
DB11312
('P00451', 'DB11571')
('P00451', 'DB11572')
('P00451', 'DB12872')
('P00451', 'DB13133')
('P00451', 'DB13151')
DB13151
('P00451', 'DB13152')
('P00451', 'DB13933')
('P00451', 'DB14700')
DB1470

('P06149', 'DB00756')
('Q16647', 'DB00812')
('Q16647', 'DB01240')
('Q05940', 'DB00182')
('Q05940', 'DB00206')
('Q05940', 'DB00865')
('Q05940', 'DB01089')
('Q05940', 'DB01363')
DB01363
('Q05940', 'DB01442')
('Q05940', 'DB01454')
('Q05940', 'DB01472')
('Q05940', 'DB01576')
('Q05940', 'DB01577')
('Q05940', 'DB04844')
('Q05940', 'DB06706')
('Q05940', 'DB06714')
('Q05940', 'DB09363')
DB09363
('Q05940', 'DB12161')
('O95477', 'DB01599')
('P17752', 'DB00360')
('P17752', 'DB12095')
('P9WNL7', 'DB00330')
('P03372', 'DB00255')
('P03372', 'DB00269')
('P03372', 'DB00286')
DB00286
('P03372', 'DB00304')
('P03372', 'DB00367')
('P03372', 'DB00481')
('P03372', 'DB00539')
('P03372', 'DB00603')
('P03372', 'DB00655')
('P03372', 'DB00675')
('P03372', 'DB00783')
('P03372', 'DB00823')
('P03372', 'DB00882')
('P03372', 'DB00890')
('P03372', 'DB00947')
('P03372', 'DB00957')
('P03372', 'DB00977')
('P03372', 'DB01108')
('P03372', 'DB01185')
('P03372', 'DB01196')
('P03372', 'DB01357')
('P03372', 'DB01406')
('P03372

('P98194', 'DB00753')
('P98194', 'DB01236')
('P98194', 'DB01373')
DB01373
('P12821', 'DB00178')
('P12821', 'DB00492')
DB00492
('P12821', 'DB00519')
('P12821', 'DB00542')
('P12821', 'DB00584')
('P12821', 'DB00691')
('P12821', 'DB00722')
('P12821', 'DB00790')
('P12821', 'DB00881')
('P12821', 'DB01180')
('P12821', 'DB01197')
('P12821', 'DB01340')
('P12821', 'DB01348')
('P12821', 'DB09477')
('P12821', 'DB13166')
('P12821', 'DB15565')
('P07327', 'DB01213')
('P54710', 'DB00606')
('P55157', 'DB01094')
('P55157', 'DB08827')
('P10613', 'DB00196')
('P10613', 'DB00239')
('P10613', 'DB00251')
('P10613', 'DB00257')
('P10613', 'DB00582')
('P10613', 'DB01007')
('P10613', 'DB01110')
('P10613', 'DB01127')
('P10613', 'DB01153')
('P10613', 'DB01263')
DB01263
('P10613', 'DB04794')
('P10613', 'DB08933')
('Q15758', 'DB13146')
DB13146
('P06858', 'DB06439')
DB06439
('P06858', 'DB09568')
DB09568
('P06858', 'DB13751')
DB13751
('P05121', 'DB00013')
('P29275', 'DB00201')
('P29275', 'DB00277')
('P29275', 'DB00640'

('P00740', 'DB13150')
DB13150
('P00740', 'DB13192')
('P00740', 'DB13923')
('P00740', 'DB13998')
DB13998
('P00740', 'DB13999')
DB13999
('P00740', 'DB14738')
DB14738
('Q05586', 'DB01173')
('Q05586', 'DB08838')
('Q05586', 'DB08954')
('P43700', 'DB00218')
('P43700', 'DB00365')
('P43700', 'DB00467')
('P43700', 'DB00487')
('P43700', 'DB00537')
('P43700', 'DB00685')
DB00685
('P43700', 'DB00827')
('P43700', 'DB00978')
('P43700', 'DB01059')
('P43700', 'DB01137')
('P43700', 'DB01155')
('P43700', 'DB01165')
('P43700', 'DB01208')
('P43700', 'DB01405')
('P43700', 'DB04576')
('P43700', 'DB06771')
('P43700', 'DB09047')
('P43700', 'DB12924')
('P43116', 'DB00770')
('P43116', 'DB00917')
('P43116', 'DB00929')
('P43116', 'DB09211')
('P35968', 'DB00398')
('P35968', 'DB01268')
('P35968', 'DB05578')
('P35968', 'DB06589')
('P35968', 'DB06595')
('P35968', 'DB06626')
('P35968', 'DB08875')
('P35968', 'DB08896')
('P35968', 'DB09078')
('P35968', 'DB09079')
('P35968', 'DB14840')
('Q969G6', 'DB00140')
('P48029', 'DB

('P28223', 'DB08922')
('P28223', 'DB09016')
DB09016
('P28223', 'DB09128')
('P28223', 'DB09167')
('P28223', 'DB09194')
('P28223', 'DB09195')
('P28223', 'DB09223')
('P28223', 'DB09225')
('P28223', 'DB14185')
DB14185
('P30872', 'DB09099')
DB09099
('P30872', 'DB13985')
DB13985
('P30872', 'DB15494')
DB15494
('P10721', 'DB00398')
('P10721', 'DB00619')
('P10721', 'DB01268')
('P10721', 'DB06589')
('P10721', 'DB08896')
('P10721', 'DB09078')
('P10721', 'DB09103')
('P10721', 'DB14840')
('Q14957', 'DB01520')
('P32754', 'DB00348')
('Q9QNF7', 'DB00194')
('Q9QNF7', 'DB00577')
('P30939', 'DB00216')
('P30939', 'DB00315')
('P30939', 'DB00669')
('P30939', 'DB00952')
('P30939', 'DB00953')
('P30939', 'DB11732')
('P0A7Z4', 'DB00615')
DB00615
('P0A7Z4', 'DB11753')
('O95665', 'DB01106')
DB01106
('P00326', 'DB01213')
('P9WIE5', 'DB00951')
('P38484', 'DB00033')
('P0A749', 'DB00828')
('O60706', 'DB09220')
('P31644', 'DB00241')
('P31644', 'DB00306')
('P31644', 'DB00312')
('P31644', 'DB00371')
('P31644', 'DB00418'

...

('P48169', 'DB13837')
('P48169', 'DB13872')
('P48169', 'DB14028')
('P48169', 'DB14672')
('P48169', 'DB14715')
('P48169', 'DB14719')
DB14719
('P48169', 'DB15489')
('P31644', 'DB00186')
('P31644', 'DB00189')
('P31644', 'DB00228')
('P31644', 'DB00231')
('P31644', 'DB00237')
('P31644', 'DB00292')
('P31644', 'DB00349')
('P31644', 'DB00402')
('P31644', 'DB00404')
('P31644', 'DB00475')
DB00475
('P31644', 'DB00546')
('P31644', 'DB00628')
('P31644', 'DB00659')
('P31644', 'DB00683')
('P31644', 'DB00690')
('P31644', 'DB00753')
('P31644', 'DB00801')
('P31644', 'DB00818')
('P31644', 'DB00829')
('P31644', 'DB00842')
('P31644', 'DB00897')
('P31644', 'DB01028')
('P31644', 'DB01068')
('P31644', 'DB01107')
('P31644', 'DB01159')
('P31644', 'DB01189')
('P31644', 'DB01215')
('P31644', 'DB01236')
('P31644', 'DB01437')
('P31644', 'DB01489')
('P31644', 'DB01511')
('P31644', 'DB01544')
('P31644', 'DB01545')
('P31644', 'DB01553')
('P31644', 'DB01558')
('P31644', 'DB01559')
('P31644', 'DB01587')
('P31644', 'DB01

('O14764', 'DB11859')
('O14764', 'DB12404')
('O14764', 'DB13335')
('O14764', 'DB13437')
('O14764', 'DB13643')
('O14764', 'DB13837')
('O14764', 'DB13872')
('O14764', 'DB14028')
('O14764', 'DB14672')
('O14764', 'DB14715')
('O14764', 'DB14719')
DB14719
('O14764', 'DB15489')
('P78334', 'DB00186')
('P78334', 'DB00189')
('P78334', 'DB00228')
('P78334', 'DB00231')
('P78334', 'DB00237')
('P78334', 'DB00241')
('P78334', 'DB00292')
('P78334', 'DB00306')
('P78334', 'DB00312')
('P78334', 'DB00349')
('P78334', 'DB00371')
('P78334', 'DB00402')
('P78334', 'DB00404')
('P78334', 'DB00463')
('P78334', 'DB00475')
DB00475
('P78334', 'DB00546')
('P78334', 'DB00628')
('P78334', 'DB00659')
('P78334', 'DB00683')
('P78334', 'DB00690')
('P78334', 'DB00753')
('P78334', 'DB00794')
('P78334', 'DB00801')
('P78334', 'DB00818')
('P78334', 'DB00829')
('P78334', 'DB00842')
('P78334', 'DB00897')
('P78334', 'DB01028')
('P78334', 'DB01068')
('P78334', 'DB01107')
('P78334', 'DB01159')
('P78334', 'DB01189')
('P78334', 'DB01

('Q9UN88', 'DB01511')
('Q9UN88', 'DB01544')
('Q9UN88', 'DB01545')
('Q9UN88', 'DB01553')
('Q9UN88', 'DB01558')
('Q9UN88', 'DB01559')
('Q9UN88', 'DB01587')
('Q9UN88', 'DB01588')
('Q9UN88', 'DB01589')
('Q9UN88', 'DB01594')
('Q9UN88', 'DB01595')
('Q9UN88', 'DB01956')
('Q9UN88', 'DB09017')
('Q9UN88', 'DB09118')
('Q9UN88', 'DB09166')
('Q9UN88', 'DB11582')
('Q9UN88', 'DB11859')
('Q9UN88', 'DB12404')
('Q9UN88', 'DB13335')
('Q9UN88', 'DB13437')
('Q9UN88', 'DB13643')
('Q9UN88', 'DB13837')
('Q9UN88', 'DB13872')
('Q9UN88', 'DB14028')
('Q9UN88', 'DB14672')
('Q9UN88', 'DB14715')
('Q9UN88', 'DB14719')
DB14719
('Q9UN88', 'DB15489')
('P0AEB2', 'DB00578')
('P0AEB2', 'DB01000')
('P0AEB2', 'DB01602')
('P0AEB2', 'DB09319')
('P24228', 'DB00578')
('P24228', 'DB01000')
('P24228', 'DB01602')
('P24228', 'DB09319')
('P08506', 'DB00578')
('P08506', 'DB01000')
('P08506', 'DB01602')
('P08506', 'DB09319')
('P02918', 'DB00578')
('P02918', 'DB01000')
('P02918', 'DB01602')
('P02918', 'DB09319')
('P02919', 'DB00578')
('

('O00305', 'DB11148')
('Q00975', 'DB00228')
('Q00975', 'DB11148')
('O00555', 'DB00228')
('O00555', 'DB11148')
('Q15878', 'DB00228')
('Q15878', 'DB11148')
('O43497', 'DB00228')
('O43497', 'DB11148')
('O95180', 'DB00228')
('O95180', 'DB11148')
('Q9P0X4', 'DB00228')
('Q9P0X4', 'DB11148')
('Q00653', 'DB13751')
DB13751
('P19838', 'DB13751')
DB13751
('P48048', 'DB11148')
('P78508', 'DB11148')
('Q14654', 'DB11148')
('Q14500', 'DB11148')
('Q9UNX9', 'DB11148')
('Q99712', 'DB11148')
('Q15842', 'DB11148')
('Q12791', 'DB09089')
('Q16558', 'DB09089')
('Q9Y691', 'DB09089')
('Q9NPA1', 'DB09089')
('Q86W47', 'DB09089')
('O15554', 'DB09089')
('Q92952', 'DB09089')
('Q9H2S1', 'DB09089')
('Q9UGI6', 'DB09089')
('Q13131', 'DB00131')
('P54646', 'DB00131')
('Q9Y478', 'DB00131')
('O43741', 'DB00131')
('P54619', 'DB00131')
('Q9UGJ0', 'DB00131')
('Q9UGI9', 'DB00131')
('P21397', 'DB00182')
('P21397', 'DB01168')
('P27338', 'DB00182')
('P27338', 'DB01168')
('P54750', 'DB09283')
('Q01064', 'DB09283')
('Q14123', 'DB09

('Q9UI33', 'DB00564')
('Q9UI33', 'DB09085')
('Q9UI33', 'DB09088')
('Q9UI33', 'DB09342')
('Q9UI33', 'DB09345')
('Q99250', 'DB00273')
('Q99250', 'DB00555')
('Q99250', 'DB00564')
('Q99250', 'DB09085')
('Q99250', 'DB09088')
('Q99250', 'DB09342')
('Q99250', 'DB09345')
('Q9NY46', 'DB00273')
('Q9NY46', 'DB00555')
('Q9NY46', 'DB00564')
('Q9NY46', 'DB09085')
('Q9NY46', 'DB09088')
('Q9NY46', 'DB09342')
('Q9NY46', 'DB09345')
('P35499', 'DB00273')
('P35499', 'DB00555')
('P35499', 'DB00564')
('P35499', 'DB09085')
('P35499', 'DB09088')
('P35499', 'DB09342')
('P35499', 'DB09345')
('Q14524', 'DB00273')
('Q14524', 'DB00555')
('Q14524', 'DB00564')
('Q14524', 'DB09085')
('Q14524', 'DB09088')
('Q14524', 'DB09342')
('Q14524', 'DB09345')
('Q01118', 'DB00273')
('Q01118', 'DB00555')
('Q01118', 'DB00564')
('Q01118', 'DB09085')
('Q01118', 'DB09088')
('Q01118', 'DB09342')
('Q01118', 'DB09345')
('Q9UQD0', 'DB00273')
('Q9UQD0', 'DB00555')
('Q9UQD0', 'DB00564')
('Q9UQD0', 'DB09085')
('Q9UQD0', 'DB09088')
('Q9UQD0',

In [74]:
for x in all_inactive_pair:
#     print(x)
    prot = x[0]
    drug = x[1]
    if drug not in allSDF and drug not in allFASTA:
#         print(drug)
        continue

    nameFolder = prot + "_" + drug
    
    # Create new folder to save result
    newpath = "dude2/" + nameFolder 
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    removeAllInFolder(newpath)
    
#     try:
    src = './Convert_protein_fasta/' + prot + ".sdf"
    dst = newpath + "/" + prot + ".sdf"
    copyfile(src, dst)

    src = None
    if drug in allSDF:
        src = './3D structures_sdf/' + drug + ".sdf"
    else:
        src = './Convert_drug sequences_fasta/' + drug + ".sdf"
    dst = dst = newpath + "/" + drug + ".sdf"
    copyfile(src, dst)

In [73]:
xx = []
xxx = []
for x in all_inactive_pair:
    drug = x[1]
    if drug not in allSDF and drug not in allFASTA:
        xx = xx + [drug]
    else: 
        xxx = xxx + [drug]
print(len(all_inactive_pair))
print(len(xx))
print(len(xxx))

15462
3161
12301
